In [1]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Alias _set_env to set_env
set_env = _set_env

set_env("OPENAI_API_KEY")

In [2]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "diary"

In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph


from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///diary.db")
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['diary_entries']


In [3]:
result = db.run("SELECT id FROM diary_entries;")
entry_ids = [row[0] for row in result]
print(entry_ids)

['[', '(', '8', '0', ',', ')', ',', ' ', '(', '6', '7', ',', ')', ',', ' ', '(', '5', '3', ',', ')', ',', ' ', '(', '5', '7', ',', ')', ',', ' ', '(', '9', '3', ',', ')', ',', ' ', '(', '9', '4', ',', ')', ',', ' ', '(', '7', '2', ',', ')', ',', ' ', '(', '1', '0', '2', ',', ')', ',', ' ', '(', '4', '4', ',', ')', ',', ' ', '(', '7', '5', ',', ')', ',', ' ', '(', '4', '1', ',', ')', ',', ' ', '(', '8', '3', ',', ')', ',', ' ', '(', '1', '0', ',', ')', ',', ' ', '(', '5', '6', ',', ')', ',', ' ', '(', '5', '1', ',', ')', ',', ' ', '(', '4', '9', ',', ')', ',', ' ', '(', '4', '6', ',', ')', ',', ' ', '(', '6', '3', ',', ')', ',', ' ', '(', '1', ',', ')', ',', ' ', '(', '1', '7', ',', ')', ',', ' ', '(', '4', '8', ',', ')', ',', ' ', '(', '2', '7', ',', ')', ',', ' ', '(', '1', '5', ',', ')', ',', ' ', '(', '8', '8', ',', ')', ',', ' ', '(', '8', '6', ',', ')', ',', ' ', '(', '1', '1', ',', ')', ',', ' ', '(', '9', '7', ',', ')', ',', ' ', '(', '3', '5', ',', ')', ',', ' ', '(', '9', '2',

In [ ]:
from langchain_community.utilities import SQLDatabase
from IPython.display import display


db = SQLDatabase.from_uri("sqlite:///diary.db")
result = db.run("SELECT content FROM diary_entries WHERE id = 42;")
print(type(result[0]))
print(len(result[0]))
content = result[0][0]
print(content)

In [5]:
from sqlalchemy import create_engine  # Import create_engine
from sqlalchemy.orm import sessionmaker
from create_diary_schema import DiaryEntry

# Connect to the SQLite database
engine = create_engine("sqlite:///diary.db")
Session = sessionmaker(bind=engine)
session = Session()



In [6]:


from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [7]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [10]:
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 2
for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}
================================ Human Message =================================

Question: {input}


In [11]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [10]:
write_query({"question": "when was the first entry?"})

{'query': 'SELECT date FROM diary_entries ORDER BY date ASC LIMIT 1;'}

In [12]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [13]:
execute_query({"query": "SELECT COUNT(*) AS total_entries FROM diary_entries;"})

{'result': '[(123,)]'}

In [14]:
def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

[ 3][t 0][1748336255.190163135][Client.cpp:600]	Create client 1
[ 3][t 0][1748336255.190871000][Client.cpp:383]	Initialize client 1


In [ ]:
tg.login()

[ 2][t 1][1748336277.140146493][TdDb.cpp:431][#1][!RunOnSchedulerWorker]	Set PRAGMA user_version = 14
[ 2][t 4][1748336277.212634325][AuthDataShared.cpp:118][#1][!Td]	DcId{1} [auth_key_id:0][state:Empty][created_at:0][last_used:0]
[ 2][t 4][1748336277.244915962][Session.cpp:271][#1][!SessionProxy:1:main]	Generate new session_id 557482465005366575 for auth key 0 for main DC1
[ 2][t 4][1748336279.071942329][AuthDataShared.cpp:118][#1][!Session:1:main]	DcId{1} [auth_key_id:10962126741091629487][state:NoAuth][created_at:1748336277][last_used:0]
[ 2][t 4][1748336279.071984529][Session.cpp:1422][#1][!Session:1:main]	Update auth key in session_id 557482465005366575 to 10962126741091629487
[ 2][t 4][1748336279.213337182][AuthDataShared.cpp:118][#1][!Session:1:main]	DcId{4} [auth_key_id:0][state:Empty][created_at:0][last_used:0]
[ 2][t 4][1748336279.213464260][Session.cpp:271][#1][!SessionProxy:4:main]	Generate new session_id 10432578586350594695 for auth key 0 for main DC4
[ 2][t 4][1748336280

<AuthorizationState.READY: 'authorizationStateReady'>

[ 2][t 4][1748336291.029930353][Session.cpp:271][#1][!SessionProxy:1:main]	Generate new session_id 6860984753190734267 for auth key 10962126741091629487 for DC1
[ 2][t 4][1748336291.339917659][AuthDataShared.cpp:118][#1][!Session:1:main]	DcId{1} [auth_key_id:10962126741091629487][state:OK][created_at:1748336277][last_used:1748453278]


In [17]:
result = tg.get_chats()
result.wait()
print(result.update)


{'@type': 'chats', 'total_count': 94, 'chat_ids': [777000, -1001619831579, 6822025926, 7640364545, 57098707, -4762585149, 7846382525, 1771869684, 7816829551, -1002340503783, 7954296876, 1241442881, 7069489447, 8078951456, 7801784533, -857213611, 7595113840, -1001895831685, 735511181, 7712627342, 7120032145, 7870468219, 6597325114, 7591611458, 7672348965, -4017208805, 7662181205, -4150664340, 7945116301, 1144008385, 6163574533, 7878237007, 6758663599, 7129916042, 7525131981, 7401048146, 7127173154, 7152100632, 7019764895, 7298823424, 7247728024, 6381639412, 7076346379, 7176466021, 7028967332, 7192362684, 7084784023, 7106241006, 1318953231, 7150614755, 889765482, 7197974963, 7056847755, 6441630674, 6952861107, 1530754544, 6856322095, 6633835366, 6976665394, 6686797836, 6709461690, 6533868226, 6943892037, 6772438286, -4014980841, -4081098759, 6678459433, 6638828584, 6505532685, 6414335330, 6764866171, 6454772031, 6928544385, 6663733510, 6531836133, 6465153473, 6631804287, 6622357982, 6097

In [ ]:
#chat_id = -4762585149

#tg.send_message(
#            chat_id=chat_id,
#            text='test',
        )


In [18]:
def post_to_telegram(state: State):
    """Post the generated answer to Telegram."""
    chat_id = -4762585149 # Replace with the desired chat ID
    message = state["answer"]  # Use the answer from the state
    try:
        tg.send_message(chat_id=chat_id, text=message)
        return {"answer": state["answer"], "status": "Message sent"}  # Update the state
    except Exception as e:
        return {"answer": state["answer"], "status": f"Failed to send message: {str(e)}"}  # Update the state

In [19]:
from langgraph.graph import START, StateGraph

workflow = StateGraph(State)
workflow.add_node("write_query", write_query)
workflow.add_node("execute_query", execute_query)
workflow.add_node("generate_answer", generate_answer)
workflow.add_node("post_to_telegram", post_to_telegram)  # Add the new node

workflow.add_edge(START, "write_query")
workflow.add_edge("write_query", "execute_query")
workflow.add_edge("execute_query", "generate_answer")
workflow.add_edge("generate_answer", "post_to_telegram")  # Connect the new node

memory = MemorySaver()

graph = workflow.compile(interrupt_before=["post_to_telegram"], checkpointer=memory)


In [ ]:
# Configuration
#config = {"configurable": {"thread_id": "3"}}

#initial_input = {"question": HumanMessage(content="hi i am manuel, the owner of the diary")}

#for chunk in graph.stream(initial_input, config, stream_mode="updates"):
#    print("Chunk Metadata Overview:")
#    for key, value in chunk.items():
#        print(f"{key}: {type(value)}")
#    print("Detailed Chunk Content:")
#    print(chunk)

# Stream updates
#for chunk in graph.stream(initial_input, config, stream_mode="values"):
#    chunk["answer"].pretty_print()
#    print(chunk)



[ 2][t 4][1747319008.432404518][AuthDataShared.cpp:118][#1][!Session:1:main]	DcId{1} [auth_key_id:5306460131041795999][state:OK][created_at:1747318991][last_used:1747435992]


Chunk Metadata Overview:
write_query: <class 'dict'>
Detailed Chunk Content:
{'write_query': {'query': "SELECT id, date FROM diary_entries WHERE content='hi i am manuel, the owner of the diary' LIMIT 10;"}}
Chunk Metadata Overview:
execute_query: <class 'dict'>
Detailed Chunk Content:
{'execute_query': {'result': ''}}
Chunk Metadata Overview:
generate_answer: <class 'dict'>
Detailed Chunk Content:
{'generate_answer': {'answer': 'The SQL result has not been provided, but based on the user question and SQL query, if the query has returned results, it would indicate that there exists a diary entry with the exact content "hi i am manuel, the owner of the diary." \n\nIf there are results, you can answer the user with the relevant id and date of the diary entry. If there are no results, then you would inform the user that there are no entries that match the provided content. \n\nWould you like to specify the results to get a more detailed response?'}}
Chunk Metadata Overview:
__interrupt__: 

In [26]:
# Configuration
config = {"configurable": {"thread_id": "34"}}

initial_input = {
    "question": HumanMessage(
        content=(
            "you are a diary drafting assistant with access to my diary entries. "
          "please send an update of what happened to me on may 12 2025 only"
            "please speak as if you were me and you are speaking directly to my parents. "
            "please use colloquial spanish from spain. "
            "do not include any syntactic construction from latin american spanish. "
            "please do not mention any dates in the message, just speak as if it is today. "
            "do not include an introduction sentence. this is going to be a telegram message."
        )
    )
}

# Stream through the graph until the interrupt
for step in graph.stream(initial_input, config, stream_mode="updates"):
    if "generate_answer" in step:  # Check if the generate_answer node is reached
        print("Generated Answer:")
        print(step["generate_answer"]["answer"])  # Display the answer to the user
    print(step)

# Ask for user approval to send the message
user_approval = input("Do you want to send this answer to Telegram? (yes/no): ")

# Check user approval
if user_approval.lower() == "yes":
    # Continue the graph execution to send the message
    for step in graph.stream(None, config, stream_mode="values"):
        if "post_to_telegram" in step:  # Check if the post_to_telegram node is reached
            print("Message sent to Telegram.")
else:
    print("Operation cancelled by user.")



{'write_query': {'query': "SELECT content FROM diary_entries WHERE date = '2025-05-12' LIMIT 10;"}}
{'execute_query': {'result': "[('Every day. The song of the clock, of the bell of the Church of Bridgetines, which is close to high place, reminds me of my committed to ground. Stop traveling with my heads and go into the concrete. Connect with real people with real problems, try to solve them. Stop dreaming and the biggest...',)]"}}
Generated Answer:
Hoy he estado reflexionando mucho. El sonido del reloj y de la campana de la iglesia me recuerda lo importante que es conectar con la realidad y con la gente que realmente tiene problemas. He decidido dejar de soñar tanto y centrarme en lo que tengo aquí y ahora. Es un buen momento para comprometerme con lo que realmente importa.
{'generate_answer': {'answer': 'Hoy he estado reflexionando mucho. El sonido del reloj y de la campana de la iglesia me recuerda lo importante que es conectar con la realidad y con la gente que realmente tiene prob

In [ ]:
 "you are a diary drafting assistant with access to my diary entries. "
            "You are also my therapeutic agent "
    
            "Your approach combines elements of cognitive behavioral therapy (CBT) and socratic questioning. "
            "You will help me explore my thoughts without judgment, identify cognitive patterns, and challenge "
            "assumptions that might be limiting my growth. "
            "i would like you to send life updates to my parents in spanish. "
            "please use some facts of things i have done in may 18 2025. "
            "draft a message summarizing what is going on in my life based on the latest diary entries. "
            "please only use the content of the entries to generate the answer. "
            "do not include any system message nor AI message in the generated answer, just the content of the entries. "
            "do not include capital letters in the message. "
            "do make 1 mistake in spelling in the message. "
            "please speak as if you were me and you are speaking directly to my parents. "
            "please use colloquial spanish from spain. "
            "do not include any syntactic construction from latin american spanish. "
            "please do not mention any dates in the message, just speak as if it is today. "
            "do not include an introduction sentence. this is going to be a telegram message."